# **Introduzione pratica ai criteri di DiVincenzo in Qiskit 2**


![](DiVincenzos.png)

*Notebook per il QiskitFallFest 2025 per il Dipartimento di Matematica e Informatica dell'Università di Catania - Fornito da IBM, tradotto e adattato da Damiano Trovato.*

## Introduzione

Nel 1996, il fisico **David DiVincenzo** definisce **cinque requisiti** che un sistema fisico **deve rispettare** per essere definito computer quantistico, oltre che a due criteri relativi alla quantum communication. In questo notebook, osserveremo e toccheremo con mano **ogni criterio di DiVincenzo tramite dimostrazioni pratiche in Qiskit**. Invece di andare nel profondo della teoria, ogni sezione spiegherà a grandi linee un criterio, fornendone poi degli esercizi da svolgere in Qiskit 2. Potrete eseguire poi i codici su simulatori o su vero hardware IBM.

**I cinque criteri di DiVincenzo per la computazione quantistica**:

1. **Un sistema fisico scalabile con qubit ben definiti**.
2. **Possibilità di inizializzare i qubit** ad uno specifico stato (ad esempio |00…0〉).
3. **Tempi di decoerenza molto lunghi** (la coerenza dei qubit deve durare molto più del tempo di operazione dei vari gate).
4. **Un set universale di porte quantistiche** (capaci di effettuare operazioni unitarie arbitrarie).
5. **Possibilità di misurare specifici qubit** (leggendone lo stato).

*(DiVincenzo descrive pure due criteri relativi alla comunicazione quantistica: 6. L'abilità di interconvertire qubit stazionari e "volanti"; 7. L'abilità di trasmettere in maniera fedele i qubit volanti tra posti fisici. Li includiamo in una parte consigliata a fine notebook.)*

Ognuna delle seguenti sezioni coincide ad un criterio. Useremo Qiskit per illustrare il concetto tramite codice e **esperimenti interattivi** che potete provare. Ad esempio, vedremo come lo scalare il numero di qubit e la profondità del circuito influenzi i risultati del circuito (Criterio 1), come resettare lo stato dei qubit (Criterio 2), come misurare i qubit su un simulatore e su un dispositivo reale (Criterio 4), come compone Qiskit i propri gate universali (Criterio 3), come coerenza finita (T₁, T₂) impatti la computazione (Criterio 5). Arrivati alla fine, avrete un'idea più approfondita di quelle che sono le implicazioni pratiche dei criteri di DiVincenzo, e di come Qiskit ne permetta la sperimentazione.

In [ ]:
# Installa i pacchetti necessari
%pip install qiskit[visualization] qiskit-ibm-runtime qiskit-aer qiskit_ibm_runtime

**Criterio 1:** *“Un sistema fisico scalabile con qubit ben definiti.”* Questo vuol dire che necessitiamo di un hardware quantistico solido per **incrementare il numero di qubit** e averne il controllo in maniera affidabile. Le proprietà di ogni qubit (livelli di energia, ratio di errori, connettività, etc.) dovrebbero essere ben note. Essenzialmente, vogliamo poter costruire circuiti quantistici più grandi senza che il sistema si rompa. Nel pratico, andando ad aumentare il numero di qubit e/o la profondità del circuito, esso diventa prone a errori e la decoerenza si accumula. Dimostrarne la *scalabilità* significa anche capire come l'aumento delle dimensioni del circuito influenzi le performance del computer quantistico.

**Obiettivo della Demo:** Usare Qiskit per dimostrare come lo scalare un circuit (nel numero di qubit o nella profondità) influenzi la fedeltà dell'output. Simuleremo uno scenario ideale, mettendono a confronto con uno rumoroso, per vedere come sistemi più grandi soccombano a decoerenza ed errori.

Per prima cosa, costruiamo un piccolo stato entangled (GHZ state) su 3 qubit, poi uno più grande su 5 qubit, per testare la scalabilità. Uno stato GHZ ad *n* qubit è $\frac{1}{\sqrt{2}}(|0...0\rangle + |1...1\rangle)$. In una simulazione ideale, misurare uno GHZ a n-qubit ha solo due possibili esiti (solo 0 o solo 1) con uguale probabilità. Compareremo l'**output ideale** ad un **output rumoroso** mentre aumentiamo il numero di qubit.

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Circuito GHZ a 3 qubit
qc3 = QuantumCircuit(3, 3)
qc3.h(0)
qc3.cx(0, 1)
qc3.cx(1, 2)
qc3.measure([0, 1, 2], [0, 1, 2])


# Circuito GHZ a 5 qubit
qc5 = QuantumCircuit(5, 5)
qc5.h(0)
qc5.cx(0, range(1, 5))    # il qubit 0 e' in entanglement con tutti gli altri
qc5.measure(range(5), range(5))

# Transpila per il simulatore
sim_backend = AerSimulator()
pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=1)
isa_qc3 = pm.run(qc3)
isa_qc5 = pm.run(qc5)

# Esegui delle simulazioni ideali (senza rumore)
sampler = Sampler(mode=sim_backend)

job3 = sampler.run([isa_qc3], shots=1024)
result3 = job3.result()
counts3 = result3[0].data.c.get_counts()

job5 = sampler.run([isa_qc5], shots=1024)
result5 = job5.result()
counts5 = result5[0].data.c.get_counts()

print("3-qubit GHZ counts (ideal):", counts3)
plot_histogram(counts3, legend=['3-qubit ideal'], figsize=(6,4))

In [ ]:
# print("5-qubit GHZ counts (ideal):", counts5)
# plot_histogram(counts5, legend=['5-qubit ideal'], figsize=(6,4))

**Risultato atteso (caso ideale):** Lo stato GHZ a 3-qubit idealmente porta a esiti al 50% `000` e al 50% `111`. Lo stato GHZ a 5-qubit porta a ~50% `00000` e 50% `11111`. Nessun altro esito spunta, in quanto perfettamente coerente ed entangled (idealmente). L'istogramma apparirà con solo due barre, una per ciascuno stato possibile.

Successivamente, vedremo cosa succede in un **ambiente rumoroso**. useremo il modello di rumore Aer di Qiskit per simulare gli errori di un dispositivo reale. Ad esempio, possiamo prendere il backend di IBM per creare un modello rumoroso che include errori sui gate, tempo limitato per i gate, rilassamento dei qubit (T₁), sfasamento (T₂), ed errori di lettura. Useremo quindi un **finto backend** che rappresenta l'IBM Quantum Brisbane device per generare un modello di rumore, e ri-eseguiremo il circuito GHZ tramite esso.

### Esercizio 1a: Simulazione con rumore
Completa il codice sottostante per simulare il circuito GHZ in un simulatore rumoroso basato sul backend `FakeBrisbane`. Questo dimostrerà come le performance degenerino in un ambiente rumoroso, più vicino a quello reale.

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

# riutilizzeremo i circuiti qc3 e qc5 e i loro risultati dalla sezione precedente.

# --- YOUR CODE HERE ---

# 1. Crea un finto backend per IBM Quantum Brisbane
###brisbane_backend = ...


# 2. Crea un AerSimulator che imiti le proprietà (il rumore...) del brisbane_backend
###noisy_sim = ...


# 3. Transpila i circuiti per il simulatore rumoroso (questo li adatta ai gate specifici del device)
###pm = ...


###isa_qc3_noisy = ...


###isa_qc5_noisy = ...


# 4. Esegui le simulazioni rumorose e otteni i risultati
###sampler = ...


###job3 = ...


###result3_noisy = ...


###counts3_noisy = ...


###job5 = ...


###result5_noisy = ...


###counts5_noisy = ...


# --- END YOUR CODE ---

# Questa parte è stata già scritta per te per stampare e plottare gli esiti:
print("3-qubit GHZ counts (noisy):", counts3_noisy)
plot_histogram(counts3_noisy, legend=['3-qubit noisy'], figsize=(6,4))

In [ ]:
print("5-qubit GHZ counts (noisy):", counts5_noisy)
plot_histogram(counts5_noisy, legend=['5-qubit noisy'], figsize=(6,4))

### Esercizio 1b: Esegui su un vero computer quantistico di IBM `[Esercizio per casa]`
Il codice sottostante verrà eseguito su un vero computer quantistico di IBM. Questo dimostrerà il decadimento delle performance su hardware reale. 

In [ ]:
# your_api_key = "deleteThisAndPasteYourAPIKeyHere"
# your_crn = "deleteThisAndPasteYourCRNHere"

# QiskitRuntimeService.save_account(
#     channel="ibm_quantum_platform",
#     token=your_api_key,
#     instance=your_crn,
#     name="fallfest-2025",
# )

# Check that the account has been saved properly
# service = QiskitRuntimeService(name="fallfest-2025")
# print(service.saved_accounts())

# We will reuse the ideal circuits qc3 and qc5 and their results from the previous cell.

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(name="fallfest-2025")
real_backend = service.least_busy(operational=True, simulator=False)
print("Running on " + real_backend.name)

pm = generate_preset_pass_manager(backend=real_backend, optimization_level=1)
isa_qc3r = pm.run(qc3)
isa_qc5r = pm.run(qc5)

sampler = Sampler(mode=real_backend)

job3r = sampler.run([isa_qc3r], shots=1024)
result3r = job3r.result()
counts3r = result3r[0].data.c.get_counts()

job5r = sampler.run([isa_qc5r], shots=1024)
result5r = job5r.result()
counts5r = result5r[0].data.c.get_counts()

print("3-qubit GHZ counts (real):", counts3r)
plot_histogram(counts3r, legend=['3-qubit real'], figsize=(6,4))

In [ ]:
print("5-qubit GHZ counts (real):", counts5r)
plot_histogram(counts5r, legend=['5-qubit real'], figsize=(6,4))

**Output atteso (rumoroso vs ideale):** Col rumore, che sia simulato o su un dispositivo reale, lo stato GHZ è **imperfetto**. Noterai infatti risultati diversi da tutti-0 e tutti-1. Su 3 qubit, invece del 100% di `000`/`111`, saranno visibili ulteriori stati (`001`, `010`, etc.) a causa di errori nei gate e decoerenza, che porterà a dei bit-flip. Su 5 qubit, l'effetto è ancora più pronunciato; più grande è il circuito (più qubit e porte CNOT) più si accumulano gli errori, con stati tutti-0 e tutti-1 meno presenti, e più stati non attesi. Questo trend mostra la sfida intrinseca alla *scalabilità*: scalando le dimensioni, mantenere alta fedeltà nei risultati diventa sempre più difficile, senza dei meccanismi di mitigazione degli errori.

**Intuizione:** Un computer quantistico scalabile necessita di preservare la correlazione quantistica nonostante la crescita del sistema. I nostri esempi dimostrano come il numero di qubit e la profondità del circuito siano inversamente proporzionali alla fedeltà del circuito, quando è presente del rumore.

## 2. Criterio 2 – Inizializzazione dei qubit

**Criterio 2:** *“Possibilità di inizializzare i qubit ad uno specifico stato (ad esempio |00…0〉).”* Tutti i qubit devono poter essere inizializzati in maniera affidabile ad uno stato di riferimento noto (tipicamente lo stato ground |0〉 per ogni qubit). L'inizializzazione è necessaria per far partire gli algoritmi su uno stato di partenza pulito algorithms. In practica, negli IBM quantum device ogni qubit è automaticamente resettato a |0〉 all'inizio di ogni esecuzione di un circuito. Qiskit fornisce anche istruzioni per resettare qubit e preparare stati personalizzati durante la computazione.

**Obiettivo della Demo:** Mostrare come inizializzare i qubit in Qiskit, sia all'inizio che a metà circuito. Dimostreremo l'uso dell'istruzione `reset` e i metodi di preparazione dello stato.

### Esercizio 2: Preparare uno stato specifico
Nel codice sottostante, completa il `QuantumCircuit` per preparare lo stato $|10\rangle$. Il qubit 0 dovrà essere allo stato $|0\rangle$ e il qubit 1 dovrebbe essere nello stato $|1\rangle$. Usa i gate e le istruzioni appropriate.

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

# Crea un circuito per inizializzare i qubit a |10> e verifica con le misurazioni
qc_init = QuantumCircuit(2, 2)

# --- YOUR CODE HERE ---

# 1. Setta il qubit 1 allo stato |1>

# 2. Resetta esplicitamente il qubit 0 allo stato |0>

# --- END YOUR CODE ---

qc_init.measure([0, 1], [0, 1])
qc_init.draw('mpl')


In [ ]:
# Esegui il circuito e plotta gli output
sim_backend = AerSimulator()
pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=1)
isa_qc_init = pm.run(qc_init)

sampler = Sampler(mode=sim_backend)

job = sampler.run([isa_qc_init], shots=1024)
result = job.result()
counts = result[0].data.c.get_counts()

print("Outcome of |10> state measured in Z-basis:", counts)
plot_histogram(counts)

Il risultato dovrebbe essere `10` (qubit1=1, qubit0=0) con probabilità 100%, e quindi il qubit 1 sarà stato messo a |1〉 e il qubit 0 a |0〉 in maniera corretta.

Ora, per una preparazione degli stati iniziali più generale, Qiskit offre un inizializzazione a stati arbitrari usando il metodo `initialize(state)`. Ad esempio, prepariamo un qubit nello stato $|+\rangle = (|0\rangle+|1\rangle)/\sqrt{2}$, che è uno stato in superposizione, e un paio di qubit nello stato di Bell $(|00\rangle+|11\rangle)/\sqrt{2}$:

In [ ]:
import numpy as np

# Inizializza un singolo qubit nello stato |+> state e misura le z-basi
qc_plus = QuantumCircuit(1, 1)
state_plus = [1/np.sqrt(2), 1/np.sqrt(2)]   # ampiezze su |0> e |1>
qc_plus.initialize(state_plus, 0)
qc_plus.measure(0, 0)

# Inizializza due qubit in uno stato di Bell manualmente
qc_bell = QuantumCircuit(2, 2)
bell_state = [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]  # amplitudes for |00>,|01>,|10>,|11>
qc_bell.initialize(bell_state, [0, 1])
qc_bell.measure([0, 1], [0, 1])

# Transpila ed esegui i circuiti di inizializzazione
isa_qc_plus = pm.run(qc_plus)
job_plus = sampler.run([isa_qc_plus], shots=1024)
result_plus = job_plus.result()
counts_plus = result_plus[0].data.c.get_counts()

print("Outcome of |+> state measured in Z-basis:", counts_plus)

isa_qc_bell = pm.run(qc_bell)
job_bell = sampler.run([isa_qc_bell], shots=1024)
result_bell = job_bell.result()
counts_bell = result_bell[0].data.c.get_counts()

print("Outcome of Bell state measured in Z-basis:", counts_bell)


**Risultato atteso:** Lo stato |+〉 del singolo qubit, quando misurato, collasserà a `0` o `1` con probabilità circa 50% per ciascuno dei possibili stati. La misurazione dello stato di Bell dovrebbe dare circa 50% `00` and 50% `11`. Se sono questi gli esiti, l'inizializzazione è avvenuta con successo.

**Inizializzazione a metà circuito:** Il metodo `reset` di Qiskit può essere utilizzato a metà circuito per reinizializzare un qubit a |0〉 al volo. Ad esempio, in codici di correzione degli errori o in algoritmi iterativi, un qubit viene spesso misurato e poi resettato per essere riutilizzato. L'operazione di `reset` è deterministica; cestina qualsiasi stato presente e porta il qubit allo stato di ground.

**Esempio su dispositivo:** Su hardware come l **ibmq_brisbane** (127 qubits) o qualsiasi dispositivo quantum di IBM, tutti i qubit partono da |0〉 di default ogni qual volta si esegua un job. Se sono necessari altri stati di partenza, si applicano gli opportuni gate (come quando abbiamo applicato il gate X per ottenere |1〉). Le reinizializzazioni continue (per la correzione degli errori) sono un argomento di ricerca molto importante, in quanto effettuarle velocemente è una grossa sfida. Fortunatamente, per usi basilari, partire dallo stato |0…0〉 è un'opzione più che valida, e abbiamo già tutti gli strumenti per passare ad altri stati.

## 3. Criterio 3 – Lunghi tempi di coerenza (Decoerenza vs Tempo di esecuzione dei gate)

**Criterio 3:** *“Tempi di decoerenza abbastanza lunghi, più lunghi del tempo di esecuzione delle porte.”* Questo fa riferimento alla necessità, da parte dei qubit, di mantenere abbastanza a lungo lo stato quantistico, in modo da poter effettuare tutte le operazioni necessarie. Ogni qubit ha **tempo T₁** (tempo di rilassamento energetico, quanto velocemente |1〉 decade a |0〉) e **tempo T₂** (tempo di sfasamento, quanto velocemente si perde la coerenza di fase). Per il funzionamento di un computer quantistico, questi lassi di tempo dovrebbero durare molto più del tempo di esecuzione delle porte.

**Obiettivo della Demo:** Investigare sulla coerenza dei qubit in Qiskit mostrando come la decoerenza impatti i risultati, soprattutto su lunghi tempi di esecuzioni. Useremo un falso backend con tempi T1/T2 noti per simulare questo effetto.

Per **dimostrare l'impatto del tempo di coerenza finito**, simuleremo un esperimento di decadenza di T1. Prepareremo un qubit nello stato |1〉, aspetteremo un po' di tempo con l'istruzione `delay`, e poi misureremo. Ci aspettiamo che la probabilità di misurare |1〉 decadi ad ogni aumento di delay.

In [ ]:
# Questa parte è stata fatta per te. Andremo a creare una lista di circuiti,
# ognuno di questi con un delay differente

time_delays_ns = [0, 50000, 100000, 150000, 200000, 250000, 300000]  # delay durations in ns

decay_expts = []
for delay in time_delays_ns:
    qc = QuantumCircuit(1, 1)
    qc.x(0)  # inizializza i qubit a |1>
    if delay > 0:
        qc.delay(delay, 0, unit='ns')  # aspetta 'delay' nanosecondi
    qc.measure(0, 0)
    decay_expts.append(qc)

decay_expts[1].draw('mpl') # visualizza uno dei circuiti

### Esercizio 3: Simulare un esperimento sulla decadenza di T1

Adesso, usiamo un simulatore rumoroso basato su `FakeVigo` (che ha tempo T1 di circa ~50-100 µs, *microsecondi*) per eseguire questi circuiti. Il simulatore applicherà automaticamente gli errori relativi a T1/T2 durante le istruzioni `delay`. Transpila il circuito per questo backend ed eseguilo.

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeVigoV2 as FakeVigo
from qiskit_aer import AerSimulator

# --- YOUR CODE HERE ---

# 1. Crea un simulatore rumoroso basato sul backend FakeVigo
###sim_vigo = ...


# 2. Transpila la lista di circuiti per questo simulatore
###pm = ...


###isa_decay_expts = ...


# 3. Usa il campionatore per eseguire tutti i circuiti transpilati assieme
###sampler = ...


###job = ...


###result = ...


# --- END YOUR CODE ---

# Questa parte è stata scritta per te per analizzare e stampare i risultati.
for idx, (delay, qc) in enumerate(zip(time_delays_ns, isa_decay_expts)):
    counts = result[idx].data.c.get_counts()
    p1 = counts.get('1', 0) / 1000  # Assumiamo 1000 esecuzioni
    print(f"Delay {delay} ns: P(qubit=1) = {p1:.3f}")


Dovresti osservare la probabilità `P(qubit=1)` decadere quando il delay aumenta, seguendo una curva di decadenza esponenziale caratteristica del rilassamento di T1. Questo dimostra direttamente come il tempo di coerenza finito porti a errori computazionali se il circuito viene eseguito per troppo tempo.

**Impatto sugli algoritmi:** Se provi algoritmi più lunghi (con molte porte in sequenza), l'esecuzione totale si avvicina e può superare T2, causando perdita di coerenza prima della fine dell'esecuzione. Ecco perché migliorare il tempo di coerenza e rendere le porte più veloci, è un punto critico nella ricerca nell'ambito hardware quantistico.

## 4. Criterio 4 – Set universale di porte quantistiche

**Criterio 4:** *“A ‘universal’ set of quantum gates.”* Il nostro hardware deve permetterci di eseguire *qualsiasi* computazione quantistica componendo un set finito di porte base. Nella computazione classica, sappiamo già che la porta NAND è universale; nel quantum computing, ci sono molti set universali di porte ({H, T, CNOT} o le porte native di ogni macchina). I dispositivi di IBM, per esempio, hanno un set di operazioni native, come operazioni di rotazione arbitrarie a singolo qubit e CNOT tra certi qubit, che assieme compongono un insieme universale. Il lavoro di Qiskit è spesso quello di **compilare gate di alto livello in gate base universali**.

**Obiettivo della Demo:** Illustrare universalità dei gate mostrando come Qiskit li decomponga. Prenderemo dei gate non-nativi (come il gate di Toffoli a 3-qubit 3, CCX) e vedremo come sono composti nei gate di base del dispositivo. Questo dimostra che il set di gate fornito è, senza ombra di dubbio, *universale* – permettendo quindi di effettuare operazioni molto complesse.

Prima, vediamo cosa sono i gate di base in un tipico backend di IBM. Otterremo la configurazione di qualsiasi dispotivo reale (o la sua versione fake), usando `.configuration().basis_gates`. Ad esempio, i gate di base del backend ibmq_brisbane sono:

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
fake_brisbane = FakeBrisbane()
print("Basis gates for ibmq_brisbane:", fake_brisbane.configuration().basis_gates)

Questo darà un output del tipo `['id', 'rz', 'sx', 'x', 'ecr']`. Queste sono le operazioni primitive che nell'hardware supporta nativamente (Identità/no-op, rotazione RZ, porta sqrt(X), porta X e X controllato). Qualsiasi altro gate sarà composto dall'unione di questi. Questo set è noto di essere universale per il quantum computing (un set universale è composto essenzialmente da rotazioni a singolo qubit + un gate di entanglement a due qubit).

Ora, prendiamo un **gate di Toffoli (CCX)** come test-case. CCX ruota un qubit solo se due qubit di controllo sono entrambi a 1. Non è un gate nativo nell'hardware di IBM. Qiskit fornisce un'istruzione `ccx`, ma dietro le quinte verrà decomposta.

### Esercizio 4: Decomponiamo un gate di Toffoli
Completa il codice sottostante per costruire un circuito con un gate di Toffoli (CCX), poi usa Qiskit per decomporlo nei gate base nativi del backend `FakeBrisbane`.

In [ ]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

# The fake_brisbane backend from the previous cell is reused here.

# --- YOUR CODE HERE ---

# 1. Crea un Quantumcircuit che possa ospitare un gate di Toffoli
###qc_toffoli = ...


# Applica il gate con 0 e 1 come bit di controllo e il qubit 2 come target


# 2. Transpila il circuito per il backend FakeBrisbane
###pm = ...


###isa_qc_toffoli = ...


# --- END YOUR CODE ---

print("Toffoli circuit before decomposition:")
print(qc_toffoli)

print("\nToffoli circuit after transpiling to Brisbane basis:")
# Il metodo .draw() mostrerà adesso il circuito decostruito
print(isa_qc_toffoli.draw(fold=120))

Nell'output transpilato, dovresti vedere il gate CCX rimpiazzato da una sequenza di gate basilari, come i gate `rz`, `sx` e `ecr`. Questo dimostra che i gate nativi sono sufficienti a replicare il gate di Toffoli.

**Universalità nel pratico:** L'esercizio appena svolto dimostra che un gate complesso come questo gate a 3-qubit sia in realtà costituito da più gate nativi. In generale, **qualsiasi** gate a più qubit può essere composto da gate a 1- e 2-qubit. Il transpilatore è quindi un componente fondamentale in ogni quantum software stack, in quanto fa da ponte tra gli algoritmi astratti che vogliamo eseguire, e le operazioni fisiche tipiche dello specifico dispositivo.

**Esempio di dispositivo:** L'**ibmq_brisbane** usa l'architettura Eagle con la lista di gate precedentemente citata. Questo significa che ogni algoritmo ad esso mandato, sarà in realtà tradotto in una sequenza di queste istruzioni. Questo criterio fa quindi riferimento alla **controllabilità**; è necessario avere abbastanza operazioni base per effettuare qualsiasi tipo di operazione complessa sui qubit.

## 5. Criterio 5 – Misurabilità dei qubit

**Criterio 5:** *“Possibilità di misurare specifici qubit”* Lo stato di ogni qubit deve essere misurabile (typicamente nella base computazionale, |0〉 or |1〉). In altre parole, dopo aver eseguito un circuito quantistico, dobbiamo poter leggere tutti i qubit come bit 0/1 classici. Questo criterio fa riferimento all'avere un modo affidabile per leggere il valore dei qubit, in modo da poter anche specificare quale qubit misurare.

**Obiettivo della Demo:** Mostrare come effettuare misurazioni a qubit specifici in Qiskit su simulatori e dispositivi reali, ed evidenziarne le differenze (come il rumore di misurazione). Misureremo alcuni qubit in vari stati ed esamineremo i risultati. Dimostreremo come gli errori di lettura possono apparire confrontando i risultati dei simulatori con quelli dell'hardware reale.

Per prima cosa, diamo un esempio di misurazione:

In [ ]:
qc_measure = QuantumCircuit(2, 2)
qc_measure.x(0)              # qubit 0 -> |1>, qubit 1 stays |0>
qc_measure.measure([0, 1], [0, 1])
qc_measure.draw('mpl')

In [ ]:
sim_backend = AerSimulator()
pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=1)
isa_qc_measure = pm.run(qc_measure)
sampler = Sampler(mode=sim_backend)
job = sampler.run([isa_qc_measure], shots=1000)
result = job.result()
counts = result[0].data.c.get_counts()

print("Simulator measurement counts:", counts)

Ci aspettiamo 1000 misurazioni di `01` nel simulatore. Ora, vediamo gli **errori di misurazione** in azione, simulandoli. Possiamo aggiungere degli errori di lettura al nostro Aer simulator. Qiskit Aer ci permette di definire un `ReadoutError` e di aggiungerlo ai qubit tramite un noise model.

### Esercizio 5: Simulare errori di lettura
Completa il codice per definire un semplice modello con errori di lettura dove ogni qubit ha una chance 2% di essere misurato in maniera errata (0 letto come 1, 1 letto come 0). Poi, esegui le misurazioni con un modello di rumore.

In [ ]:
from qiskit_aer.noise import NoiseModel, ReadoutError

# --- YOUR CODE HERE ---

# 1. Definisci un errore di lettura del 2% per ogni qubit.
# Il formato è una lista di liste di probabilità: [[P(0|0), P(1|0)], [P(0|1), P(1|1)]]
# P(A|B) is the probability of measuring A given the state was |B>.
###ro_error = ...

# 2. Crea un nuovo modello di rumore
###noise_model_ro = ...



# 3. Aggiungi l'errore di lettura a tutti i qubit
... # consiglio: Usa il metodo add_all_qubit_readout_error


# --- END YOUR CODE ---

sim_backend.set_options(noise_model=noise_model_ro)
pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=1)
isa_qc_measure = pm.run(qc_measure)

# Esegui la misurazione del circuito con il rumore normale
sampler = Sampler(mode=sim_backend)

job = sampler.run([isa_qc_measure], shots=1024)
result = job.result()
counts = result[0].data.c.get_counts()

print("Simulation with 2% readout error:", counts)

Questo output simulato mostrerà degli errori di simulazione (`11`, `00`, `10`) simili a ciò che l'hardware reale potrebbe produrre, dimostrando l'impatto delle misurazioni imperfette.

**Esempio su dispositivo:** Su un dispositivo reale, come l'**ibmq_brisbane**, puoi eseguire lo stesso circuito e misurare simili valori non-zero, chiaramente sbagliati. I dati relativi alla calibrazione del dispositivo indicano errori di lettura su qualsiasi qubit. Riuscire a isolare e misurare singoli qubit, e capire come si presentano gli errori, è fondamentale per ottenere dati significativi. L'esecuzione su hardware reale è stata dimostrata nell'**Esercizio 1b**.

## Criteri di comunicazione quantistica (qubit volanti)

DiVincenzo ha inoltre definito due criteri fondamentali relativi alla quantum communication, importanti da considerare nella costruzione di quantum computer interconnessi:

6. **Capacità di interconvertire qubit stazionari e volanti.** (mappare un qubit in un processore ad un fotone che può viaggiare.)
7. **Capacità di trasmettere fedelmente qubit volanti tra posti diversi.** (mandare un fotone qubit attraverso la fibra senza perdere informazione quantistica.)

Questi vanno oltre gli utilizzi standard di Qiskit, in quanto questo viene principalmente usato per lavorare con qubit stazionari su dei chip. Possiamo tuttavia illustrare il concetto dietro questi criteri attraverso un esempio: **il teletrasporto quantistico**. Il teletrasporto mostra la conversione dello stato di un qubit stazionario, trasportata da un qubit volante, grazie al loro entanglement. Questo permette di ricostruire un qubit stazionario da un'altra parte.

### Attività consigliata: *Quantum Teleportation*

Il modulo *Qiskit in Classrooms* sul [Quantum Teleportation](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/quantum-teleportation) dal Dr. Katie McCormick vi guiderà in uno dei protocolli più accattivanti di quantum information: il teletrasporto quantistico, dove uno stato quantistico (un qubit) è mandato da Alice a Bob usando l'entanglement e solo due bit classici. Imparerete l'intera procedura di teletrasporto step-by-step— come creare la coppia nello stato di Bell in entanglement, effettuare un misuramento nella base di Bell-basis dal lato di Alice, trasmettere risultato classico, e applicare le porte quantistiche opportune sul qubit dal lato di Bob, in modo da ottenere lo stato originale. Nel processo, scoprirete anche perché il teletrasporto delle informazioni relative a un qubit non viola il teorema di non clonazione, e non supera la velocità della luce. Tramite esercizi svolti su hardware quantistico di IBM o simulatori, avrete modo di vedere in azione le misurazioni, l'entanglement, e il feed-forward control.

Capendo bene il teletrasporto quantistico, capirete come codificare, trasmettere, e recuperare informazione quantistica tra nodi distinti, mettendo così, le basi per le reti quantistiche, sistemi di repeater, schemi di comunicazione sicura, e computazione quantistica scalabile e modulare.

**Come ciò si relaziona ai criteri 6 e 7:** In una vera rete quantistica, la coppia condivisa in entanglement è creata distribuendo qubit "volanti" (come i fotoni) tra Alice e Bob (Criterio 7: trasmissione affidabile). Il protocollo di teletrasporto in se, offre un modo per mappare il qubit stazionario di Alice in ad una metà di una coppia in entanglement, nell'effettivo "mandandolo" a Bob (Criterio 6: interconversione). Qiskit ci permette di simulare la logica del protocollo perfettamente, fornendo dei modelli concettuali per come questi criteri sono soddisfatti nelle architetture di comunicazione.

## Conclusioni e sommario

Abbiamo definito una serie di esercizi di programmazione in Qiskit per illustrare i criteri di DiVincenzo. Tramite questi esempio, abbiamo esplorato come una vera piattaforma di quantum computing soddisfa ciascun requisito:

- **Scalabilità**: costruire circuiti su più qubit e capire la crescita del rumore.
- **Inizializzazione**: usare reset e preparazione degli stati per avviare la computazione in maniera affidabile a stati noti.
- **Gate Universali**: transpilare operazioni complesse in gate di base dell'hardware, dimostrando di poter svolgere qualsiasi computazione.
- **Misurazione**: leggere il valore dei qubit e gestire errori di lettura realistici.
- **Coerenza**: vedere l'effetto della durata finita di T₁, T₂ sulla fedeltà dei risultati degli algoritmi e il bisogno di avere tempi di operazione molto veloci rispetto al tempo di decoerenza.

Per completezza, abbiamo toccato alcuni aspetti di comunicazione quantistica in Qiskit nel corso in [Quantum Teleportation](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/quantum-teleportation), associandoli anche ai due criteri associati (qubit volanti).


È importante sapere come questi criteri entrino in gioco in veri computer quantistici come quelli di IBM. In dispositivo come **ibmq_brisbane** ha 127 qubit superconduttori (Criterio 1), che partono sempre dallo stato |0〉 (Criterio 2), con un set di porte calibrate e compiatori per ottenere universalità (Criterio 4), risonatori a microonde per leggere ciascun qubit (Criterio 5), e tempi di coerenza dell'ordine delle centinaia di microsecondi contro le operazioni, che richiedono nano secondi (Criterio 3). Per esperimenti nelle reti quantistiche , IBM e altri stanno esplorando la trasduzione microonde-a-ottico per i qubit volanti, e l'entanglement tra qubit distanti (Criteri 6 e 7); queste sono aree di ricerca molto attive.

Completando gli esercizi di questo notebook, non solo avete visto le definizioni relative ai criteri di DiVincenzo, ma le avete *toccate con mano* attraverso il codice; costruendo anche intuizioni relative alle implicazioni sull'hardware reale e sugli algoritmi, di ciascun requisito. Estendete liberamente questi esperimenti, e felice quantum computing!